# Demo for AIFlow class


There is magic in the class - everytime you do a text prompt, the last result is stored in the context map under `latest`. So you can use that one as context variable for further processing or reference in queries.

In fact, the class uses some other magical variables stored in the context. Such as latest file loaded (if no name is provided) and token usage. When you run `show_self_data()` you can see all data magically stored in the `context_map`.

## The demo

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
API_KEY=os.getenv("API_KEY")
from aiflow import AIFlow


# Just an empty flow

In [8]:
flow1 = AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=0)
(
    flow1
    .show_model_config()
    .display_internal_data()
    .pretty_print_messages()
    .pretty_print_messages_to_file(file_name="nothing.txt")
)


Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 0
Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}


# Chat demo

In [3]:
chatbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
chatbot.show_model_config().display_internal_data()

Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 1
Chat Messages:
[]

Context Map:
{}

Images Map:
{}

Audio Map:
{}


In [4]:
(    chatbot
        .show_model_config()
        .set_system_prompt("You are a helpful cooking assistant always eager to give a recipe for whatever the user says.")
        .add_user_chat("How is the weather here?")
        .display_internal_data()
        .get_token_usage()
)

Model: gpt-3.5-turbo
Max Tokens: 3000
Temperature: 1
You are a helpful cooking assistant always eager to give a recipe for whatever the user says.

How is the weather here?
I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?

Chat Messages:
[
    {
        "role": "system",
        "content": "You are a helpful cooking assistant always eager to give a recipe for whatever the user says."
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    }
]

Context Map:
{
    "latest_prompt": "How is the weather here?",
    "_completion_tokens": 22,
    "_prompt_tokens": 35,
    "_total_tokens": 57,
    "latest": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
}

Images Map:
{}

Audio Map:
{}


{'completion_tokens': 22, 'prompt_tokens': 35, 'total_tokens': 57}

In [5]:
chatbot.add_user_chat("Ok. Give me a recipe inspired by weather")

Ok. Give me a recipe inspired by weather
How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:

Ingredients:
- 4 cups cubed watermelon
- 1 cup crumbled feta cheese
- 1/4 cup chopped fresh mint leaves
- 2 tablespoons balsamic glaze
- Salt and pepper to taste

Instructions:
1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.
2. Add the chopped fresh mint leaves and gently toss everything together.
3. Drizzle the balsamic glaze over the salad and season with salt and pepper to taste.
4. Give it a final toss to mix all the ingredients well.
5. Serve the Watermelon Feta Salad chilled and enjoy as a light and refreshing dish on a hot day!

Enjoy your Watermelon Feta Salad inspired by the weather!



In [6]:
chatbot.copy_latest_to("dish")

In [7]:
chatbot.display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are a helpful cooking assistant always eager to give a recipe for whatever the user says."
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everyth

## Message manipulations

In [8]:
chatbot.reduce_messages_to_text(lambda x: (print(x), "Replacing latest with all sorts of nothing")[1]).display_internal_data()

[{'role': 'system', 'content': 'You are a helpful cooking assistant always eager to give a recipe for whatever the user says.'}, {'role': 'user', 'content': 'How is the weather here?'}, {'role': 'assistant', 'content': "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"}, {'role': 'user', 'content': 'Ok. Give me a recipe inspired by weather'}, {'role': 'assistant', 'content': "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the salad and season with salt and pepper to taste.\n4. Give it a final toss to mix all the

In [9]:
chatbot.filter_messages(lambda items: [{**item, "content": "You are an artist very good in poetry"} if item.get("role") == "system" else item for item in items])

In [10]:
chatbot.display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the sa

## Just completions - no chat

In [11]:
chatbot.generate_completion("Give me 10 random names")

Give me 10 random names

1. Stella
2. Elijah
3. Penelope
4. Marcus
5. Isabella
6. Declan
7. Harper
8. Felix
9. Jade
10. Owen



In [12]:
chatbot.display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the sa

In [13]:
chatbot.generate_completion("Give the following names in reverse order: [latest].")

Give the following names in reverse order: [latest].

10. Owen
9. Jade
8. Felix
7. Harper
6. Declan
5. Isabella
4. Marcus
3. Penelope
2. Elijah
1. Stella



## Saving and loading state
To avoid having the rerun all queries when you restart the kernel etc..

In [14]:
chatbot.save_internal_state(filename="mystate.json")

In [15]:
chatbot.load_internal_state(filename="mystate.json").display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the sa

## Some context manipulations

In [16]:
chatbot.display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the sa

In [17]:
chatbot.set_context_of(content="Nothing", label="stuff").display_internal_data().delete_context(label="stuff").display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the sa

## Dumping content to files

In [18]:
chatbot.dump_context_to_files().dump_context_to_markdown()

In [19]:
chatbot.generate_heading_for_context(label="dish")


In [20]:
chatbot.save_context_to_docx(output_filename="output.docx")

In [21]:
chatbot.display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the sa

In [22]:
chatbot.get_token_usage()

{'completion_tokens': 306, 'prompt_tokens': 381, 'total_tokens': 687}

## Data loading

In [23]:
chatbot.load_to_context(filename="output.txt").display_internal_data()

Chat Messages:
[
    {
        "role": "system",
        "content": "You are an artist very good in poetry"
    },
    {
        "role": "user",
        "content": "How is the weather here?"
    },
    {
        "role": "assistant",
        "content": "I'm happy to give you a recipe instead. What kind of dish are you in the mood for today?"
    },
    {
        "role": "user",
        "content": "Ok. Give me a recipe inspired by weather"
    },
    {
        "role": "assistant",
        "content": "How about making a refreshing Watermelon Feta Salad that's perfect for hot weather? Here's a simple recipe for you:\n\nIngredients:\n- 4 cups cubed watermelon\n- 1 cup crumbled feta cheese\n- 1/4 cup chopped fresh mint leaves\n- 2 tablespoons balsamic glaze\n- Salt and pepper to taste\n\nInstructions:\n1. In a large bowl, combine the cubed watermelon and crumbled feta cheese.\n2. Add the chopped fresh mint leaves and gently toss everything together.\n3. Drizzle the balsamic glaze over the sa

# Combining AIFlows

In [24]:
otherbot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
otherbot.generate_completion("Let's say you are a random person hungry for food. What would you like to eat?")

Let's say you are a random person hungry for food. What would you like to eat?

As a language model AI, I do not have the ability to eat or cravings like a regular person. But I can certainly help suggest some popular food options, such as pizza, pasta, burgers, sushi, tacos, or a sandwich. Let me know if you would like more suggestions!



In [25]:
otherbot.display_latest_context_as_markdown()

'As a language model AI, I do not have the ability to eat or cravings like a regular person. But I can certainly help suggest some popular food options, such as pizza, pasta, burgers, sushi, tacos, or a sandwich. Let me know if you would like more suggestions!'

In [11]:
sillybot=AIFlow(api_key=API_KEY,model="gpt-3.5-turbo",max_tokens=3000, temperature=1)
sillybot.set_context_of(content=otherbot.return_latest_to_text(), label="food_choice").display_internal_data()
sillybot.generate_completion(prompt="Give a silly comment on the following choice of food: [food_choice]").display_internal_data()

NameError: name 'otherbot' is not defined

# Using vectordatabase Chroma

In [27]:
# https://docs.trychroma.com/guides
import chromadb
chroma_client = chromadb.Client() # chromadb.PersistentClient(path="/path/to/save/to")
collection = chroma_client.create_collection(name="my_collection")

In [28]:
collection.add(
    documents=[ 
        "This is a document about pineapple",
        "This is a document about oranges",
        "This is a document about cars",
        "This is a document about pears",
        "This is a document about computers",
        "This is a document about fruit",
        "This is a document about tropical fruits",
    ],
    ids=["id1", "id2", "id3", "id4", "id5", "id6", "id7"],
)

In [29]:
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=3 # how many results to return
)
print(results)

{'ids': [['id1', 'id7', 'id6']], 'distances': [[1.0404009819030762, 1.0958093404769897, 1.2092766761779785]], 'metadatas': [[None, None, None]], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about tropical fruits', 'This is a document about fruit']], 'uris': None, 'data': None}


In [30]:
collection.peek() # returns a list of the first 10 items in the collection
collection.count() # returns the number of items in the collection

7

In [31]:
from aiflow import chroma_query_result_to_text
print(chroma_query_result_to_text(results))

This is a document about pineapple
This is a document about tropical fruits
This is a document about fruit


In [32]:
sillybot.set_context_of(content=chroma_query_result_to_text(results), label="found_doc")
sillybot.generate_completion("Say something really silly about this random text: [found_doc]").show_self_data()

Say something really silly about this random text: [found_doc]

This document is so fruity, it's about to start singing "Strawberry Fields Forever!"

Chat Messages:
[]

Context Map:
{
    "food_choice": "As a language model AI, I do not have the ability to eat or cravings like a regular person. But I can certainly help suggest some popular food options, such as pizza, pasta, burgers, sushi, tacos, or a sandwich. Let me know if you would like more suggestions!",
    "latest_prompt": "Say something really silly about this random text: This is a document about pineapple\nThis is a document about tropical fruits\nThis is a document about fruit",
    "_completion_tokens": 63,
    "_prompt_tokens": 113,
    "_total_tokens": 176,
    "latest": "This document is so fruity, it's about to start singing \"Strawberry Fields Forever!\"",
    "found_doc": "This is a document about pineapple\nThis is a document about tropical fruits\nThis is a document about fruit"
}

Images Map:
{}

Audio Map:
{}


# Working with media

https://platform.openai.com/docs/api-reference/images/create

In [43]:
flow1.generate_image(
    size="256x256", prompt="A big house in the desert.", html=False
).show_self_data().dump_image_to_file(filename="generated_output/an_image.jpg")

# URLs with images expire in 1 hour
flow1.generate_image(size="256x256", prompt="A tomato plant on a car.", html=True)

Usage ImagesResponse(created=1719425332, data=[Image(b64_json=None, revised_prompt=None, url='https://oaidalleapiprodscus.blob.core.windows.net/private/org-qiwjaBjm9TZ58jKWInrRESeh/user-krnk9YsqIV2WoGdKtr7UH5kB/img-5tms4HKsUMvSoapHnKYmxe4t.png?st=2024-06-26T17%3A08%3A52Z&se=2024-06-26T19%3A08%3A52Z&sp=r&sv=2023-11-03&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-06-25T23%3A16%3A55Z&ske=2024-06-26T23%3A16%3A55Z&sks=b&skv=2023-11-03&sig=DVol%2BEqdVLf6/KGO5/smjfMdJ657Jnv%2BQ8n2xjo500I%3D')])
Chat Messages:
[]

Context Map:
{
    "latest_image_prompt": "A big house in the desert.",
    "latest_revised_image_prompt": null
}

Images Map:
{
    "latest_image": "https://oaidalleapiprodscus.blob.core.windows.net/private/org-qiwjaBjm9TZ58jKWInrRESeh/user-krnk9YsqIV2WoGdKtr7UH5kB/img-5tms4HKsUMvSoapHnKYmxe4t.png?st=2024-06-26T17%3A08%3A52Z&se=2024-06-26T19%3A08%3A52Z&sp=r&sv=2023-11-03&sr=b&rscd=inline&rsct=image/png

https://platform.openai.com/docs/guides/vision

In [44]:
flow1.analyze_image(
    image="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
)

Usage ChatCompletion(id='chatcmpl-9eRNVrTTLJKUlE0lcJxqMPiXIf85T', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The image shows a beautiful natural landscape with a wooden boardwalk extending through a grassy field. The sky is mostly clear with some scattered clouds, and the sun appears to be providing bright, warm light, giving a peaceful and serene feel to the scene. There are trees and shrubs in the background, adding to the lush, green environment.', role='assistant', function_call=None, tool_calls=None))], created=1719425389, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_4008e3b719', usage=CompletionUsage(completion_tokens=67, prompt_tokens=97, total_tokens=164))
ChatCompletionMessage(content='The image shows a beautiful natural landscape with a wooden boardwalk extending through a grassy field. The sky is mostly clear with some scattered clouds, and the sun appears to be providing bright, warm li

https://platform.openai.com/docs/api-reference/audio/createSpeech

In [45]:
flow1.generate_speech(prompt="Jump for joy!", filename="generated_output/jump.mp3")

Usage <openai._legacy_response.HttpxBinaryResponseContent object at 0x0000018FAC3182E0>


https://platform.openai.com/docs/api-reference/audio/createTranscription

In [47]:
flow1.get_token_usage()
flow1.transcribe_audio(filename="generated_output/jump.mp3").get_token_usage()

Usage Jump for joy.

Jump for joy.



{'completion_tokens': 0, 'prompt_tokens': 0, 'total_tokens': 0}

https://platform.openai.com/docs/guides/moderation/overview

In [9]:
flow1.moderate_content("This is stupid!")

{
  "id": "modr-9eRs6hC8QI4P4zwzrFRmMRZRT5wGW",
  "model": "text-moderation-007",
  "results": [
    {
      "categories": {
        "harassment": false,
        "harassment/threatening": false,
        "hate": false,
        "hate/threatening": false,
        "self-harm": false,
        "self-harm/instructions": false,
        "self-harm/intent": false,
        "sexual": false,
        "sexual/minors": false,
        "violence": false,
        "violence/graphic": false,
        "self-harm": false,
        "sexual/minors": false,
        "hate/threatening": false,
        "violence/graphic": false,
        "self-harm/intent": false,
        "self-harm/instructions": false,
        "harassment/threatening": false
      },
      "category_scores": {
        "harassment": 0.013359030708670616,
        "harassment/threatening": 2.8022158815588227e-8,
        "hate": 0.00011000907397828996,
        "hate/threatening": 5.6863641399729214e-12,
        "self-harm": 8.968489062510798e-9,
      

# Youtube download, transcription and summary

Let's transcribe and ask to summarise (will take a while)

In [4]:
from aiflow import Model
from pytube import YouTube

(
    chatbot.execute_function(
        YouTube("https://www.youtube.com/watch?v=iMo5-4h_XwE")
        .streams.get_audio_only()
        .download(filename="generated_output/zirp.mp3")
    )
    .execute_function(print("Download completed - generating transcript"))
    .transcribe_audio(filename="generated_output/zirp.mp3", label="transcript")
    .set_model(Model.GPT_4O_MINI)
    .generate_completion(
        "Summarise the following text for me:\n[transcript]", label="summary_transcript"
    )
    .dump_context_to_file(label="summary_transcript", filename="generated_output/zirp.txt")
)

Download completed - generating transcript
transcript Uh-oh. One of the sinkholes, so to speak, that the money could go into is the asset class known as venture capital. And sinkhole it is. All right. This is Dalton plus Michael. And today we're going to talk about what is ZERP and why did it mess with the startup world? I love the term ZERP. Explain to everyone what ZERP is, the actual term. ZERP is one of those words that I find myself using a fair amount, that I often will be asked to pause and explain, what was that word you just said? And so we are saying the word ZERP, Z-I-R-P. Yeah, you got to say it like that. You got to say it like that. Z-I-R-P, and that is an acronym, right? For Zero Interest Rate Phenomenon. Yes. That's the period we lived through that ended as COVID was fatal. So in this time, the interest rate set by the Federal Reserve in the United States was zero. Yes. Some banks in Europe, you had to pay them. Yes, they had negative interest rates. And so look, we're 

# The end

In [12]:
flow1.save_internal_state(filename="flow1_state.json")
chatbot.save_internal_state(filename="chatbot_state.json")
sillybot.save_internal_state(filename="sillybot_state.json")